## 項目 76 関係する振る舞いを TestCase サブクラスで検証する

Python でテストを書く正しい標準的な方法は、組み込みモジュール unittest を使うことである。

例えば、utils.py で定義された、次のようなユーティリティ関数を、さまざまな入力に対して正しく動作するか検証しようとする。


In [3]:
## utils.py ##
def to_str(data):
    if isinstance(data, str):
        return data
    elif isinstance(data, bytes):
        return data.decode('utf-8')
    else:
        raise TypeError('Must supply str or bytes, '
                        'found: %r' % data)

テストを定義するために、 期待する各振る舞いのテストを含んだ test_utils.py または utils_test.py という名の第2のファイルを作る (ファイル名は任意) 。

In [6]:
## utils_test.py ##
from unittest import TestCase, main
# from utils import to_str

class UtilsTestCase(TestCase):
    def test_to_str_bytes(self):
        self.assertEqual('hello', to_str(b'hello'))

    def test_to_str_str(self):
        self.assertEqual('hello', to_str('hello'))

    def test_failing(self):
        self.assertEqual('incorrect', to_str('hello'))

# if __name__ == '__main__':
#     main()


テストファイルを Python コマンドラインで実行する。この場合、テストメソッドの2つは無事パスするが、1つは失敗し、何がいけなかったのかを示すエラーメッセージが表示される。

```
utils_test.py
F..
======================================================================
FAIL: test_failing (__main__.UtilsTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\utils_test.py", line 28, in test_failing
    self.assertEqual('incorrect', to_str('hello'))
AssertionError: 'incorrect' != 'hello'
- incorrect
+ hello


----------------------------------------------------------------------
Ran 3 tests in 0.001s

FAILED (failures=1)
```

テストは、TestCaseサブクラスで構成する (上の例では UtilsTestCase)。各テストケースは、test_ という語で始まるメソッドである。テストメソッドが何の Exception (assert 文の AssertionError も含めて) も起こさずに実行できた場合、テストは成功したと考えられる。1つのテストが失敗した場合、TestCase サブクラスは最初の問題点で停止するのではなく、継続して他のテストメソッドを実行し、テスト全体の様子がわかるようにする。

もし、特定のテストで迅速にバグを修正したり、改善を反復したい場合には、コマンドラインのテストモジュールで、そのパスを指定して、そのテストメソッドだけを実行することができる。


```
$ python3 utils_test.py UtilsTestCase.test_to_str_bytes
.
---------------------------------------------------------
Ran 1 test in 0.000s

OK
```

テストメソッドの中で、特定のブレークポイントで障害の原因をより深く調べるために直接デバッガを呼び出すこともできる (どのように行うかは「項目 80 pdbで対話的にデバッグすることを考える」参照)。

TestCase クラスは、テストで確認のためのアサーションを行うためのヘルパー関数を提供する。等価性を検証する assertEqual、論理式を検証する assertTrue など多数ある (他は help(TestCase) 参照)。これらは、入出力すべてを出力して、テストが失敗した理由をしっかりと理解できるため、組み込みの assert 文よりも有益である。例えば次に、ヘルパーアサーションメソッドがある場合とない場合とで同じテストケースを比較する。


In [ ]:
from unittest import TestCase, main
from utils import to_str

class AssertTestCase(TestCase):
    def test_assert_helper(self):
        expected = 12
        found = 2 * 5
        self.assertEqual(expected, found)

    def test_assert_statement(self):
        expected = 12
        found = 2 * 5
        assert expected == found

# if __name__ == '__main__':
#     main()

次の失敗メッセージを比較すると分かるよう、ヘルパー関数 assertionEqual を用いたケースの方が原因が判別しやすいだろう。

```
assert_test.py
FF
======================================================================
FAIL: test_assert_helper (__main__.AssertTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\assert_test.py", line 24, in test_assert_helper
    self.assertEqual(expected, found)
AssertionError: 12 != 10
```

```
======================================================================
FAIL: test_assert_statement (__main__.AssertTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\assert_test.py", line 29, in test_assert_statement
    assert expected == found
AssertionError

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=2)
```

with 文でコンテキストマネジャーとして使うことができる例外を検証する assertRaises ヘルパーメソッドもある(どのように動作するかは「項目 66 contextlib と with 文を try/finally の代わりに考える」参照)。これは、try/except 文とよく似ており、どこで例外が発生すると期待されるかを明確にする。


In [7]:
## utils_error_test.py ##
from unittest import TestCase, main

class UtilsErrorTestCase(TestCase):
    def test_to_str_bad(self):
        with self.assertRaises(TypeError):
            to_str(object())

    def test_to_str_bad_encoding(self):
        with self.assertRaises(UnicodeDecodeError):
            to_str(b'\xfa\xfa')

# if __name__ == '__main__':
#     main()


```
python utils_error_test.py
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK
```

テストをより読みやすくするために、複雑なロジックを持つ専用のヘルパーメソッドを TestCase サブクラスで定義できる。ただし、メソッド名が test という語から始まらないようにしないと、テストケースと間違われて実行されてしまうので、その点は要確認である。こういったカスタマイズしたテストヘルパーでは、TestCase アサーションを呼び出すだけでなく、しばしば fail メソッドを使って、どこで仮定や不変条件が成り立たなくなったかを明らかにする。

例えば、ジェネレータの振る舞いを検証するカスタムヘルパーメソッドを次のように定義する。

In [12]:
## helper_test.py ##
from unittest import TestCase, main

def sum_squares(values):
    cumulative = 0
    for value in values:
        cumulative += value ** 2
        yield cumulative

class HelperTestCase(TestCase):
    def verify_complex_case(self, values, expected):
        expect_it = iter(expected)
        found_it = iter(sum_squares(values))
        test_it = zip(expect_it, found_it)

        for i, (expect, found) in enumerate(test_it):
            self.assertEqual(
                expect,
                found,
                f'Index {i} is wrong')

        # Verify both generators are exhausted
        try:
            next(expect_it)
        except StopIteration:
            pass
        else:
            self.fail('Expected longer than found')

        try:
            next(found_it)
        except StopIteration:
            pass
        else:
            self.fail('Found longer than expected')

    def test_wrong_lengths(self):
        values = [1.1, 2.2, 3.3]
        expected = [
            1.1**2,
        ]
        self.verify_complex_case(values, expected)

    def test_wrong_results(self):
        values = [1.1, 2.2, 3.3]
        expected = [
            1.1**2,
            1.1**2 + 2.2**2,
            1.1**2 + 2.2**2 + 3.3**2 + 4.4**2,
        ]
        self.verify_complex_case(values, expected)

# if __name__ == '__main__':
#     main()


ヘルパーメソッドは、テストケースを短く読みやすくし、出力されるエラーメッセージがわかりやすくしている。

```
python helper_test.py
FF
======================================================================
FAIL: test_wrong_lengths (__main__.HelperTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\helper_test.py", line 57, in test_wrong_lengths
    self.verify_complex_case(values, expected)
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\helper_test.py", line 50, in verify_complex_case
    self.fail('Found longer than expected')
AssertionError: Found longer than expected

======================================================================
FAIL: test_wrong_results (__main__.HelperTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\helper_test.py", line 66, in test_wrong_results
    self.verify_complex_case(values, expected)
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\helper_test.py", line 32, in verify_complex_case
    self.assertEqual(
AssertionError: 36.3 != 16.939999999999998 : Index 2 is wrong

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=2)
```

通常、関係するテストセットごとに1つの TestCase サブクラスを定義する。
時には、多くのテストケースを持つ関数ごとに1つの TestCase サブクラスということもある。
他のケースとしては、TestCase サブクラスが単一モジュールのすべての関数を処理することもある。各基底クラスとそのすべてのメソッドをテストする TestCase サブクラスを作ることもしばしばある。

TestCase クラスは、1つのテストメソッドの中で複数のテストを定義するときに、ボイラープレート (使い回す前提で用意されたテキスト) を避けるための subTest ヘルパーメソッドも提供する。これは特に、データ駆動テストを書く場合に役立つ。これは、(多数のテストメソッドを含む TestCase の振る舞いと同様に) ケースのいずれかで失敗しても他のケースのテストを継続するテストメソッドの実行を許可する。

次のデータ駆動テストの例で示す。


In [11]:
## data_driven_test.py ##
from unittest import TestCase, main

class DataDrivenTestCase(TestCase):
    def test_good(self):
        good_cases = [
            (b'my bytes', 'my bytes'),
            ('no error', b'no error'),  # This one will fail
            ('other str', 'other str'),
        ]
        for value, expected in good_cases:
            with self.subTest(value):
                self.assertEqual(expected, to_str(value))

    def test_bad(self):
        bad_cases = [
            (object(), TypeError),
            (b'\xfa\xfa', UnicodeDecodeError),
        ]
        for value, exception in bad_cases:
            with self.subTest(value):
                with self.assertRaises(exception):
                    to_str(value)

# if __name__ == '__main__':
#     main()

no error テストは失敗し、有用なエラーメッセージを出力するが、他のすべてのケースは引き続きテストされ、パスすることが確認された。


```
python data_driven_test.py
.
======================================================================
FAIL: test_good (__main__.DataDrivenTestCase) [no error]
----------------------------------------------------------------------
Traceback (most recent call last):
  File "c:\Users\su10_\python\effectivepython-master\example_code\item_76\testing\data_driven_test.py", line 29, in test_good
    self.assertEqual(expected, to_str(value))
AssertionError: b'no error' != 'no error'

----------------------------------------------------------------------
Ran 2 tests in 0.001s

FAILED (failures=1)
```

Note:

プロジェクトの複雑さとテストの要求にもよるが、 pytest (https://pytest.org/) オープン ソースパッケージとその膨大なコミュニティプラグインが特に役立つ。

### 覚えておくこと

- 組み込みモジュール unittest の TestCase クラスのサブクラスでテストを定義でき、テストしたい振る舞いごとにメソッドを定義できる。TestCase クラスにおけるテストメソッドは、test という語から始まらなければならない。
- TestCase クラスで定義された assertEqual のようなさまざまなヘルパーメソッドを使い、組み込みの assert 文を使う代わりに、テストにおける期待する振る舞いを確認できる。
- subTest ヘルパーメソッドを使い、ボイラープレートを減らしてデータ駆動テストを書くことを考える。


## 項目 77 setUp, tearDown, setUpModule, tearDownModule で他からテストを分離する


TestCaseクラス (「項目76 関係する振る舞いを TestCase サブクラスで検証する」参照)には、テストメソッドを実行する前に、テスト環境をセットアップしなければならないことがよくある。テスト環境は「テストハーネス」とも呼ばれる。これをするために、TestCase サブクラスの setUp と tearDown メソッドをオーバーライドできる。この2つのメソッドは、各テストメソッドの前後で呼び出され、「各テスト実行が隔離されている」ことを確認できる。

例えば、各テストの前に一時的なディレクトリを作成して、テストが終了後はその内容を削除する TestCase を次のように定義できる。

In [2]:
## environment_test.py ##
from pathlib import Path
from tempfile import TemporaryDirectory
from unittest import TestCase, main

class EnvironmentTest(TestCase):
    def setUp(self):
        self.test_dir = TemporaryDirectory()
        self.test_path = Path(self.test_dir.name)

    def tearDown(self):
        self.test_dir.cleanup()

    def test_modify_file(self):
        with open(self.test_path / 'data.bin', 'w') as f:
            f.write('hello')

# if __name__ == '__main__':
#     main()


In [ ]:
'''
testing/environment_test.py
.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
'''

プログラムが複雑になると、コードを隔離してテストするのではなく、モジュール間の相互作用を検証する追加のテストが欲しくなるだろう (モックのようなツールを使う。「項目78 モックを使って依存性が複雑なコードをテストする」参照)。これは、ユニットテストと統合テストとの違いである。Python では、両方の種類のテストが、モジュールが実際に一緒に動くかどうか確かめるまでは何の保証もないという、まったく同じ理由のために重要なのである。

よくある問題は、統合テストのためのテスト環境構築が計算が高価で、かなりの時間を要することである。例えば、統合テストを実行するためには、データベースプロセスを開始して、インデックスがロードされるのを待たないといけない。この種のレイテンシーがあるために、TestCase クラスの setUp と tearDown メソッドで、すべてのテストの準備と後始末を行うことが実現的ではなくなるのである。

このような状況に対応するため、unittest モジュールではモジュールレベルのテストハーネス（テスト環境）初期化をサポートしている。高価な資源を一時に構成して、全 TestCase クラスとそのテストソッドを初期化を繰り返さずに実行する。モジュールの全テストが終了したら、テストハーネスを破棄する。この振る舞いを使い、TestCase クラスを含むモジュール内で関数 setUpModule と tearDownModule を次のように定義する。


In [4]:
## integration_test.py ##
from unittest import TestCase, main

def setUpModule():
    print('* Module setup')

def tearDownModule():
    print('* Module clean-up')

class IntegrationTest(TestCase):
    def setUp(self):
        print('* Test setup')

    def tearDown(self):
        print('* Test clean-up')

    def test_end_to_end1(self):
        print('* Test 1')

    def test_end_to_end2(self):
        print('* Test 2')

# if __name__ == '__main__':
#     main()


In [ ]:
'''
example_code/item_77/testing/integration_test.py
* Module setup
* Test setup
* Test 1
* Test clean-up
.* Test setup
* Test 2
* Test clean-up
.* Module clean-up

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
'''

setUpModule が unittest で、setUp メソッドの呼び出し前一度だけ実行されていることが分かる。同様に、tearDownModule は tearDown メソッド呼び出し後に実行されている。

### 覚えておくこと

- (分離された機能の) ユニットテストと (他と相互作用するモジュールの) 統合テストの両方を書くことが重要だ。
- setUp と tearDownメソッドを用いて、テストが互いに分離され、 クリーンなテスト環境 になっていることを確かにする。
- 統合テストでは、 setUpModule と tearDownModule モジュールレベル関数を用いて、テストモジュールのそれが含むすべての TestCase クラスに必要なテストハーネスを管理する。


## 項目 78 モックを使って依存性が複雑なコードをテストする

In [10]:
# Reproduce book environment
import random
random.seed(1234)

import logging
from pprint import pprint
from sys import stdout as STDOUT

# Write all output to a temporary directory
import atexit
import gc
import io
import os
import tempfile

TEST_DIR = tempfile.TemporaryDirectory()
atexit.register(TEST_DIR.cleanup)

<bound method TemporaryDirectory.cleanup of <TemporaryDirectory 'C:\\Users\\su10_\\AppData\\Local\\Temp\\tmpy6gekdmo'>>

テストを書くときに (「項目76 関係する振る舞いを TestCase サブクラスで検証する」参照) よく使われるもう1つのやり方は、モック関数やクラスを使って、実際のものを使うのが難しすぎたり、遅すぎたりする振る舞いを模倣することである。

例えば、動物園で動物に餌をやるスケジュール管理のプログラムが必要だとする。ある種類の動物すべてのデータベースにクエリを送って、直近の食餌時刻を取得する関数を次のように定義する。


In [8]:
# Example 1
class DatabaseConnection:
    def __init__(self, host, port):
        pass

class DatabaseConnectionError(Exception):
    pass

def get_animals(database, species):
    # Query the Database
    raise DatabaseConnectionError('Not connected')
    # Return a list of (name, last_mealtime) tuples

この関数のテストのためのデータベースコネクションは、どうすればいいだろうか。試しに1つ作り、関数に渡してテストしてみる。

In [33]:
# Example 2
try:
    database = DatabaseConnection('localhost', '4444')
    
    get_animals(database, 'Meerkat')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_28368\1029646582.py", line 5, in <cell line: 2>
    get_animals(database, 'Meerkat')
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_28368\3323467131.py", line 11, in get_animals
    raise DatabaseConnectionError('Not connected')
DatabaseConnectionError: Not connected


データベースは実行されておらず、失敗しまっている。解決法の1つは、データベースサーバを実際に立てて、テスト用のコネクションを張ることである。しかし、データベースを始動し、スキーマを構成し、データを取得するなどという作業を完全に自動化するには、膨大な作業が必要である。そのうえ、データベースサーバのセットアップにも時間を要するため、ユニットテストのペースが落ち、保守が困難になる。

より良い方式は、データベースのモックを作ることである。モックは、依存関数に期待される応答を期待される呼び出しに対して与える。ここで、モックとフェイクとを混同しないことが重要です。フェイクは、DatabaseConnection クラスのほとんどの振る舞いを提供するが、永続性を持たない基本的なインメモリ、1スレッドによるデータベースのような簡易実装のことである。


Python には、モックを作成してテストに使うための組み込みモジュール unittest.mockがある。実際にデータベースにコネクションを張らずに、get_animals 関数を模倣する Mock インスタンスを次のように定義する。


In [34]:
# Example 3
from datetime import datetime
from unittest.mock import Mock

mock = Mock(spec=get_animals)
expected = [
    ('Spot', datetime(2019, 6, 5, 11, 15)),
    ('Fluffy', datetime(2019, 6, 5, 12, 30)),
    ('Jojo', datetime(2019, 6, 5, 12, 45)),
]
mock.return_value = expected

Mock クラスはモック関数を作る。モックの return_value 属性は呼び出されたときに返す値である。spec 引数は、モックがそのオブジェクト（この場合は関数）のように動作し、間違った方法で使われた場合にはエラーになることを示す。

例えば、モック関数を属性を備えたモックオブジェクトであるかのように扱ったとする。


In [35]:
# Example 4
try:
    mock.does_not_exist
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_28368\3758337726.py", line 3, in <cell line: 2>
    mock.does_not_exist
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 634, in __getattr__
    raise AttributeError("Mock object has no attribute %r" % name)
AttributeError: Mock object has no attribute 'does_not_exist'


作成した後は、モックを呼び出し、戻り値を取得し、それが期待通りか検証できる。database 引数には object を使う。モックで実際に何かするわけではなく、必要なのは、database 引数が、動作に DatabaseConnection インスタンスが必要な依存関数に正しく渡されることだけだからである (見張り object インスタンスを使った別の例は「項目55 スレッド間の協調作業にはQueue を使う」参照)。

In [36]:
# Example 5
database = object()
result = mock(database, 'Meerkat')
assert result == expected

In [37]:
mock(database, 'Meerkat')

[('Spot', datetime.datetime(2019, 6, 5, 11, 15)),
 ('Fluffy', datetime.datetime(2019, 6, 5, 12, 30)),
 ('Jojo', datetime.datetime(2019, 6, 5, 12, 45))]

これでモックが正しく応答したことが検証できるが、モックを呼び出したコードが正しい引数を指定したかどうかはどうすればわかるだろうか。このために、Mock クラスには、assert_called_once_with メソッドがあり、正確に与えられた引数で呼び出しているか検証できる。 

In [30]:
# Example 6
mock.assert_called_once_with(database, 'Meerkat')

もし、引数が間違っていれば例外が発生し、このアサーションが使われたTestCaseが失敗になる

In [15]:
# Example 7
try:
    mock.assert_called_once_with(database, 'Giraffe')
except:
    logging.exception('Expected')
else:
    assert False


ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_28368\851613133.py", line 3, in <cell line: 2>
    mock.assert_called_once_with(database, 'Giraffe')
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 931, in assert_called_once_with
    return self.assert_called_with(*args, **kwargs)
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 919, in assert_called_with
    raise AssertionError(_error_message()) from cause
AssertionError: expected call not found.
Expected: mock(<object object at 0x000001E50F8C4190>, 'Giraffe')
Actual: mock(<object object at 0x000001E50F8C4190>, 'Meerkat')


厳密には、 どの database オブジェクトでも使えるように、 一部の引数が使えるのであれば、定数 unittest.mock.ANY を、どんな引数でも取れることを示す値として使う。Mock の assert_called_with メソッドを使い、モックへの直前の呼び出し (この場合には複数回呼び出されている) が期待通りか検証する。


In [16]:
# Example 8
from unittest.mock import ANY

mock = Mock(spec=get_animals)
mock('database 1', 'Rabbit')
mock('database 2', 'Bison')
mock('database 3', 'Meerkat')

mock.assert_called_with(ANY, 'Meerkat')

テスト対象の振る舞いで、引数がそれほど問題にならない場合には、ANYが便利である。指定が多すぎるテストで、引数に対する期待値をいろいろと調べる場合よりは、指定の少ないテストで、ANY をもっと自由に使って、エラーを検出することが役立つ場合がよくある。

Mock クラスでは、例外をモックで送出することも簡単である。


In [17]:
# Example 9
try:
    class MyError(Exception):
        pass
    
    mock = Mock(spec=get_animals)
    mock.side_effect = MyError('Whoops! Big problem')
    result = mock(database, 'Meerkat')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_28368\2499428456.py", line 8, in <cell line: 2>
    result = mock(database, 'Meerkat')
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 1104, in __call__
    return self._mock_call(*args, **kwargs)
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 1108, in _mock_call
    return self._execute_mock_call(*args, **kwargs)
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 1163, in _execute_mock_call
    raise effect
MyError: Whoops! Big problem


他にも多くの機能については、help (unittest.mock.Mock) を参照。

Mock の動作する仕組みを述べたので、実際にテストする状況で、どう使えばユニットテストを効果的に書くことができるかを示す。一連のデータベース関数が与えられたとして、動物園で動物たちに餌を与える関数を定義する。

In [18]:
# Example 10
def get_food_period(database, species):
    # Query the Database
    pass
    # Return a time delta

def feed_animal(database, name, when):
    # Write to the Database
    pass

def do_rounds(database, species):
    now = datetime.datetime.utcnow()
    feeding_timedelta = get_food_period(database, species)
    animals = get_animals(database, species)
    fed = 0

    for name, last_mealtime in animals:
        if (now - last_mealtime) > feeding_timedelta:
            feed_animal(database, name, now)
            fed += 1

    return fed

テストの目的は、do_rounds の実行において、餌を与える動物を間違えていないか、直近の食餌時刻をデータベースに記録したか、関数が返す餌を与えた動物の総数が正しいかを検証することである。これらすべてを行うために、datetime.utcnow のモックを作り、夏時間やその他一時的な変更の影響を受けない安定した時刻を得る必要がある。get_food_period と get_animals のモックを作り、データベースから得る値を返す必要もある。feed_animal のモックを作り、データベースに書き戻す値を受け取る必要もある。

ここでの問題は、これらの関数のモックを作り、期待する動作を設定するやり方がわかったとしても、実際の代わりのモック依存関数を使ってテストされる do_rounds 関数はどうすればいいのか、ということである。解決策の1つは、すべてをキーワード専用引数で与えるというものである (項目25 キー ワード専用引数と位置専用引数で明確さを高める」参照)。


In [19]:
# Example 11
def do_rounds(database, species, *,
              now_func=datetime.utcnow,
              food_func=get_food_period,
              animals_func=get_animals,
              feed_func=feed_animal):
    now = now_func()
    feeding_timedelta = food_func(database, species)
    animals = animals_func(database, species)
    fed = 0

    for name, last_mealtime in animals:
        if (now - last_mealtime) > feeding_timedelta:
            feed_func(database, name, now)
            fed += 1

    return fed

この関数をテストするには、Mock インスタンスをすべて前もって作成しておき、期待を設定する必要がある。


In [20]:
# Example 12
from datetime import timedelta

now_func = Mock(spec=datetime.utcnow)
now_func.return_value = datetime(2019, 6, 5, 15, 45)

food_func = Mock(spec=get_food_period)
food_func.return_value = timedelta(hours=3)

animals_func = Mock(spec=get_animals)
animals_func.return_value = [
    ('Spot', datetime(2019, 6, 5, 11, 15)),
    ('Fluffy', datetime(2019, 6, 5, 12, 30)),
    ('Jojo', datetime(2019, 6, 5, 12, 45)),
]

feed_func = Mock(spec=feed_animal)

それから、モックを do_rounds 関数に渡し、デフォルト値をオーバーライドする。

In [21]:
# Example 13
result = do_rounds(
    database,
    'Meerkat',
    now_func=now_func,
    food_func=food_func,
    animals_func=animals_func,
    feed_func=feed_func)

assert result == 2

最後に、依存関数へのすべての呼び出しが期待通りだったか検証する。

In [22]:
# Example 14
from unittest.mock import call

food_func.assert_called_once_with(database, 'Meerkat')

animals_func.assert_called_once_with(database, 'Meerkat')

feed_func.assert_has_calls(
    [
        call(database, 'Spot', now_func.return_value),
        call(database, 'Fluffy', now_func.return_value),
    ],
    any_order=True)

datetime.utcnowモックの引数やそれが何回呼び出されたかは、関数の戻り値で間接的に検証されるため、検証しない。get_food_period と get_animals については、assert_called_once_with で引数を検証する。

feed_animal 関数では、unittest.mock.call ヘルパーと assert_has_calls メソッドを使っ て、データベースに書き出す呼び出しが2回、順序は関係なし、あったことを検証する。

モックをキーワード専用引数を使って与えるこの方式はうまくいくが、出力が多くテストする関数すべてに変更を加える必要がある。unittest.mock.patch 関数族では、モックを与えるのがより簡単になる。上に定義したデータベースアクセス関数のようなモジュールやクラスの属性を一時的に再割り付けする。例えば、patch を使うと get_animals をモックでオーバーライドする。


In [23]:
# Example 15
from unittest.mock import patch

print('Outside patch:', get_animals)

with patch('__main__.get_animals'):
    print('Inside patch: ', get_animals)

print('Outside again:', get_animals)

Outside patch: <function get_animals at 0x000001E50F948A60>
Inside patch:  <MagicMock name='get_animals' id='2083320455392'>
Outside again: <function get_animals at 0x000001E50F948A60>


patchは、多くのモジュール、クラス、属性に使える。with 文 (項目 66 contextlib と with文 をtry/finally の代わりに考える」参照) の中や関数デコレータ (「項目26 functools.wraps を使って関数デコレータを定義する」参照) として、あるいは TestCase クラスの setUpと tearDown メソッド (「項目76 関係する振る舞いをTestCase サブクラスで検証する」参照) の中で使えます。他のオプションについては、help(unittest.mock patch) を参照してください。 

ただし、あらゆる場合に patch がうまくいくわけではない。例えば、do_rounds をテス
トするには、datetime.utcnow クラスメソッドで返される現在時刻をモックする必要がある。
datetimeクラスがC拡張モジュールで定義されているので、Python ではこのような方式では変更できない。


In [24]:
# Example 16
try:
    fake_now = datetime(2019, 6, 5, 15, 45)
    
    with patch('datetime.datetime.utcnow'):
        datetime.utcnow.return_value = fake_now
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 1546, in __enter__
    setattr(self.target, self.attribute, new_attr)
TypeError: cannot set 'utcnow' attribute of immutable type 'datetime.datetime'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\su10_\AppData\Local\Temp\ipykernel_28368\3056307278.py", line 5, in <cell line: 2>
    with patch('datetime.datetime.utcnow'):
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 1559, in __enter__
    if not self.__exit__(*sys.exc_info()):
  File "c:\Users\su10_\anaconda3\envs\develop\lib\unittest\mock.py", line 1565, in __exit__
    setattr(self.target, self.attribute, self.temp_original)
TypeError: cannot set 'utcnow' attribute of immutable type 'datetime.datetime'


この問題を回避するためには、patch 可能な時刻を取得するヘルパー関数を作成して対処する。

In [25]:
# Example 17
def get_do_rounds_time():
    return datetime.datetime.utcnow()

def do_rounds(database, species):
    now = get_do_rounds_time()

with patch('__main__.get_do_rounds_time'):
    pass

別の方式としては、datetime.utcnow モックにはキーワード専用引数を使い、他のモックすべてにpatchを使うこともできる。

In [26]:
# Example 18
def do_rounds(database, species, *, utcnow=datetime.utcnow):
    now = utcnow()
    feeding_timedelta = get_food_period(database, species)
    animals = get_animals(database, species)
    fed = 0

    for name, last_mealtime in animals:
        if (now - last_mealtime) > feeding_timedelta:
            feed_animal(database, name, now)
            fed += 1

    return fed

- 19

最後の方式を使うことにする。patch.multiple関数を使って多数のモックを作り期待を設定する。

- 20
  
セットアップができたので、テストを実行して、patch.multiple を使った with文の中で呼び出しが正しいか検証する。


In [29]:
# Example 19
from unittest.mock import DEFAULT

with patch.multiple('__main__',
                    autospec=True,
                    get_food_period=DEFAULT,
                    get_animals=DEFAULT,
                    feed_animal=DEFAULT):
    now_func = Mock(spec=datetime.utcnow)
    now_func.return_value = datetime(2019, 6, 5, 15, 45)
    get_food_period.return_value = timedelta(hours=3)
    get_animals.return_value = [
        ('Spot', datetime(2019, 6, 5, 11, 15)),
        ('Fluffy', datetime(2019, 6, 5, 12, 30)),
        ('Jojo', datetime(2019, 6, 5, 12, 45))
    ]

# Example 20
    result = do_rounds(database, 'Meerkat', utcnow=now_func)
    assert result == 2

    get_food_period.assert_called_once_with(database, 'Meerkat')
    get_animals.assert_called_once_with(database, 'Meerkat')
    feed_animal.assert_has_calls(
        [
            call(database, 'Spot', now_func.return_value),
            call(database, 'Fluffy', now_func.return_value),
        ],
        any_order=True)

patch.multipleのキーワード引数は、テスト中にオーバーライドする__main__モジュールでの 名前に対応する。DEFAULT値は、その名前に対しては標準Mockインスタンス作成を要求すること
を示す。生成されたモックすべては、引数 autospec=True により、模倣するオブジェクトの仕様に従う。

これらのモックは期待通りに動作するが、テストの可読性をさらに改善して、コードをリファクタリングし、余分なボイラープレートを減らし、よりテストしやすくすることが可能なことを理解しておくことが重要である (「項目79 モックとテストを活用して依存性をカプセル化する」参照)。


### 覚えておくこと

- unittest.mockモジュールは、Mock クラスを使ってインタフェースの振る舞いを模倣する方法を提供する。テストするコードで要求される依存性をセットアップするのが困難な場合に、モックがテストに役立つ。
- モックを使う場合、テストするコードの振る舞いとそのコードで呼び出される依存関数の呼び出し方の両方を、Mock.assert_called_once_withメソッド群を使って検証することが重要だ。
- キーワード専用引数と unittest.mock.patch 関数族を使って、テストするコードにモックを投入できる。